*Copyright 2023 The Specials Authors. Licensed under the Apache License, Version 2.0 (the "License").*

_Some of the code in this file is adapted from:_

_modularml/mojo_<br>
_Copyright (c) 2023, Modular Inc._<br>
_Licensed under the Apache License v2.0 with LLVM Exceptions._

# The log-beta function in Specials

This notebook presents the implementation of the **log-beta function** in Specials and assesses its quality in terms of both numerical accuracy and stability, as well as computational performance.

The log-beta function is the natural logarithm of the [beta function](https://en.wikipedia.org/wiki/Beta_function) $\text{B}$, which is a special function defined by the integral

$$\text{B}(x,y) = \int_{0}^{1} t^{x-1}(1-t)^{y-1} \,dt$$

for real numbers $x, y > 0$. The beta function is renowned for its applications in mathematics, physics, probability, and statistics. Recently, there has been a growing interest in the AI applications of this function for building deep probabilistic models with random variables following the [beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) or the [Student's _t_-distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution). In these cases, the beta function is used, for example, to define [likelihood functions](https://en.wikipedia.org/wiki/Likelihood_function) or to compute [implicit reparameterization gradients](https://arxiv.org/abs/1805.08498).

A key property of the beta function is its close relationship to the [gamma function](https://en.wikipedia.org/wiki/Gamma_function) $\Gamma$:

$$\text{B}(x,y) = \frac{\Gamma(x)\Gamma(y)}{\Gamma(x+y)} .$$

This relationship allows us to express the log-beta in terms of the log-gamma function:

$$\ln \text{B}(x,y) = \ln \Gamma(x) + \ln \Gamma(y) - \ln \Gamma(x+y) .$$

For numerical reasons and because the beta function grows very rapidly for moderately large arguments, one usually works with the latter equation instead of directly evaluating the beta function.

We call **naive solution** of the log-beta function an implementation based solely on this latter equation. This solution presents accuracy loss when at least one of its arguments takes a large value. The reason is [catastrophic cancellation](https://en.wikipedia.org/wiki/Catastrophic_cancellation) in the numerical evaluation of the difference

$$\ln \Gamma(y) - \ln \Gamma(x+y) .$$

Assume, without loss of generality (since log-beta is symmetric), that the condition $x \leq y$ holds. When $y$ is large compared to $x$, this difference tends to the difference between two almost identical numbers, which extinguishes most significant digits [[2](#machler)]. It's important to note that although $\ln \Gamma(z)$ grows less rapidly than $\Gamma(z)$ as $z$ increases, $\ln \Gamma(z)$ still behaves proportionally to $z\ln(z) - z$ asymptotically.

Specials and other scientific packages avoid this numerical cancellation by explicitly decomposing $\ln \Gamma(z)$, for sufficiently large $z$, into the approximation

$$\ln \Gamma(z) \approx (z - 0.5)\ln z - z + 0.5 \ln (2 \pi)$$

and a correction term (which we'll call **log-gamma correction**), and then cancelling the large terms from the
approximation analytically.

Before moving forward, let's install, if necessary, all the Python packages that will be used in this notebook.

In [1]:
%%python
from importlib.util import find_spec
import shutil
import subprocess

fix = """
-------------------------------------------------------------------------
To fix this, follow the steps in the link below:
    https://github.com/modularml/mojo/issues/1085#issuecomment-1771403719
-------------------------------------------------------------------------
"""

def install_if_missing(name: str):
    if find_spec(name.replace("-", "_")):
        return

    print(f"The package `{name}` was not found. We will install it...")
    try:
        if shutil.which("python3"): python = "python3"
        elif shutil.which("python"): python = "python"
        else:
            raise RuntimeError("Python is not on `PATH`. " + fix)
        subprocess.check_call([python, "-m", "pip", "install", name])
    except:
        raise ImportError(f"The package `{name}` was not found. " + fix)

install_if_missing("mpmath")
install_if_missing("numpy")
install_if_missing("scipy")
install_if_missing("tabulate")
install_if_missing("tensorflow")
install_if_missing("tensorflow-probability")


## 1. Approximating the log-gamma correction

To compute the log-gamma correction, Specials provides the function `lgamma_correction`. Since there is no analytical solution for this correction, the function evaluates it using a [Chebyshev approximation](https://mathworld.wolfram.com/ChebyshevApproximationFormula.html). We will introduce the Chebyshev approximation later in this section.


The function `lgamma_correction` is defined for $x \geq 8$, where $x$ is a single or double precision floating-point argument.

In [2]:
import specials

let x = SIMD[DType.float32, 4](7.0, 8.0, 9.0, 10.0)
let res = specials.lgamma_correction(x)

print(res)


[nan, 0.010411262512207031, 0.0092554632574319839, 0.0083305658772587776]


Note that `lgamma_correction` returns `nan` when $x < 8$.

To compare with the values returned by the Specials function, we'll call the corresponding function from the [TensorFlow Probability](https://www.tensorflow.org/probability) package using the same input values.


In [3]:
%%python
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import numpy as np
from tensorflow_probability.substrates import numpy as tfp

x = np.array([8.0, 9.0, 10.0], dtype=np.float32)
res = tfp.math.log_gamma_correction(x)

print(res)


[0.01041127 0.00925546 0.00833056]


Now let's introduce the Chebyshev approximation with an illustrative example. Let $T_k(x)$ be the Chebyshev polynomial of the first kind with degree $k$, $k = 0, 1, \dots$, and let $f$ be a real function defined in the interval $[-1, 1]$ such that $f(x) = \exp(x)$.

For $N = 15$, we use the Python packages [`mpmath`](https://mpmath.org/) and [NumPy](https://numpy.org/) to find the coefficients $c_k, k = 0, \dots, N - 1$, such that the approximation

$$f(x) \approx \sum_{k=0}^{N-1} c_kT_k(x),$$

known as the Chebyshev approximation, is exact for $x$ equal to all $k$ zeros of the $T_k$ polynomial. All zeros of $T_k$ are located in the interval $[-1, 1]$, where the function $f(x)$ is defined by construction. The estimated maximum absolute error for this approximation is about $5\times10^{-17}$.



We use Specials to generate a Chebyshev series with the coefficients obtained earlier. The class representing a Chebyshev series in Specials is designed to support various manipulations in the Mojo parameter space during compile time. Moreover, this class performs bounds and type checks during compilation, eliminating operations and preventing errors at runtime. For example, it is not possible to compile code that provides a quantity of coefficients different from the number of terms in the series or attempts to access a nonexistent coefficient.

In [4]:
from specials.polynomial import Chebyshev, chebyshev_eval, chebyshev_init

# This Chebyshev series has 15 terms (it's a polynomial with degree 14).
alias p = Chebyshev[15, DType.float32, simd_width=4].from_coefficients[
    1.266065877752008335598244625215e-00,
    1.130318207984970054415392055220e-00,
    2.714953395340765623657051399900e-01,
    4.433684984866380495257149525980e-02,
    5.474240442093732650276168431186e-03,
    5.429263119139437503621478103037e-04,
    4.497732295429514665469032791685e-05,
    3.198436462401990505863863657617e-06,
    1.992124806672795725956775710929e-07,
    1.103677172551734430729047687106e-08,
    5.505896079673739316799448763965e-10,
    2.497956616981807165974951489258e-11,
    1.039152229471141123219881783361e-12,
    3.991259006494906568717874813903e-14,
    1.422277830768448851326979168181e-15,
]()

print("p.num_terms =", p.num_terms)
print("p.degree() =", p.degree())
print("p.get[0]() =", p.get[0]())


p.num_terms = 15
p.degree() = 14
p.get[0]() = [1.2660658359527588, 1.2660658359527588, 1.2660658359527588, 1.2660658359527588]


A key property of the Chebyshev approximation is that it can be truncated to a polynomial with lower degree $m \ll N-1$ that is very close to the [minimax polynomial](https://mathworld.wolfram.com/MinimaxPolynomial.html): among all polynomials of the same degree, the minimax polynomial is the one that has the smallest maximum deviation from the true function.

Using Specials, we can find the smallest value $m$ such that the maximum deviation between the original approximating polynomial of degree $N-1$ and its truncated version with degree $m$ is upper bounded by a requested accuracy.

At compile time, we can then truncate the original Chebyshev series and later utilize the truncated version to approximate the true function $f(x)$, thereby saving operations when evaluating the approximation for various $x$ values at runtime.


In [5]:
from specials._internal.limits import FloatLimits

alias requested_accuracy = FloatLimits[DType.float32].epsneg
print("requested_accuracy =", requested_accuracy)

alias num_terms = chebyshev_init[
    p.num_terms, p.dtype, p.simd_width, p, requested_accuracy
]()

alias p_truncated = p.truncate[num_terms]()
print("p_truncated.num_terms =", p_truncated.num_terms)
print("p_truncated.degree() =", p_truncated.degree())


requested_accuracy = 5.9604644775390625e-08
p_truncated.num_terms = 9
p_truncated.degree() = 8


See how the truncated approximating polynomial returns values that apparently do not differ from those we can obtain by directly calling the standard library function `math.exp`.


In [6]:
import math

let x = SIMD[DType.float32, 4](-1.0, 0.0, 0.5, 1.0)
let res = chebyshev_eval(p_truncated, x)

print(res)
print(math.exp(x))


[0.36787945032119751, 1.0, 1.6487212181091309, 2.7182817459106445]
[0.36787945032119751, 1.0, 1.6487212181091309, 2.7182817459106445]


## 2. Evaluating the log-beta function implementation

To calculate the value of the log-beta function for real numbers $x, y$, Specials provides the `lbeta` procedure:

In [7]:
let x = SIMD[DType.float64, 4](0.1, 8.0, 30.0, 1e10)
let y = SIMD[DType.float64, 4](3.0, 1.0, 10.0, 1e-10)
let res = specials.lbeta(x, y)

print(res)


[2.1584847490202885, -2.0794415416798362, -22.572893813393978, 23.025850927580152]


When the arguments of `lbeta` are not positive and finite, the returned value is the same as that of the corresponding procedure in the R language:

In [8]:
let inf = math.limit.inf[DType.float64]()
let nan = math.nan[DType.float64]()
let x = SIMD[DType.float64, 8](-1.0, -5.0, inf, 1.0, nan, 0.0, 1.0, inf)
let y = SIMD[DType.float64, 8](10.0, -2.5, inf, nan, 1.0, 0.0, inf, 1.0)
let res = specials.lbeta(x, y)

print(res)


[nan, nan, -inf, nan, nan, inf, -inf, -inf]


Four functions, defined for internal use in the Specials package, play fundamental roles in this experiment, and for this reason, we present them here:
* `elementwise`: Applies a mathematical operator to one or more tensors element-wise, in a vectorized and potentially parallelized manner. Currently, it only supports binary operators: tensor-tensor and tensor-scalar.
* `random_uniform`: Creates a new tensor whose elements are uniformly sampled from the closed interval `[min_value, max_value]`.
* `run_benchmark`: Benchmarks the mathematical operator passed as a parameter using the `elementwise` function.
* `tensor_to_numpy_array`: Converts a Mojo tensor into a NumPy array.

In [9]:
from specials._internal.tensor import (
    elementwise,
    random_uniform,
    run_benchmark,
    tensor_to_numpy_array,
)


We demonstrate the first two of these four functions in the cell below:

In [10]:
let x = random_uniform[DType.float64](1.0, 2.0, 2)
let y = random_uniform[DType.float64](1.0, 2.0, 2)
let res = elementwise[math.add](x, y)

print(res)


Tensor([[2.3504969739512394, 3.1375148487639053]], dtype=float64, shape=2)


In this section, we evaluate `specials.lbeta` in terms of numerical **accuracy** (closeness between the result computed by `lbeta` and the true expected result) and **stability** (the ability of the procedure to produce consistent and reliable results in various input scenarios). Additionally, we examine its computational performance, measured by the execution time.


### 2.1. Experimental Settings

#### 2.1.1. Domains

To assess whether the implementation of the log-beta function in Specials produces consistent and reliable results in various input scenarios, we uniformly sampled 50,000 values for its arguments from 5 (five) intervals of the form $(0, b]$, referred to as _domains_, where $b = 10^1, \dots, 10^5$.

In this experiment, we exclusively worked with double-precision floating-point arguments (`float64`).

In [11]:
from utils.static_tuple import StaticTuple

alias dtype = DType.float64

let min_value = FloatLimits[dtype].eps
let max_values = StaticTuple[5, FloatLiteral](10.0, 100.0, 1_000.0, 10_000.0, 100_000.0)
let num_samples = 50_000


#### 2.1.2. Evaluation Metrics

To measure the accuracy of a log-beta function implementation, we use the _relative error_. Let $\hat{x}$ be an approximation of the real number $x$. The relative error $E_{\text{rel}}(\hat{x})$ is given by:

$$E_{\text{rel}}(\hat{x}) = \frac{|x - \hat{x}|}{|x|}.$$

Given the arguments, the exact but unknown value of the log-beta function, represented here by the real number $x$, is computed with high precision using the Python library [`mpmath`](https://mpmath.org/).

To compare different implementations in terms of accuracy, we calculate the maximum and mean values of the relative error for each combination of implementation and domain. Lower values indicate higher accuracy.

For quantifying computational performance, we measure the _execution time_: in Mojo, using the `benchmark` module, and in Python, by defining a function based on the `timeit` module.

To compare different implementations in terms of computational performance, we calculate the mean execution time for each combination of implementation and domain. Smaller results indicate better performance.

In [12]:
from python import Python
from python.object import PythonObject

from specials._internal.tensor import BinaryOperator

Python.add_to_path(".")


fn solution_report[
    solution_name: StringLiteral,
    func: BinaryOperator,
    dtype: DType,
    simd_width: Int = simdwidthof[dtype](),
](x: Tensor[dtype], y: Tensor[dtype], truth: PythonObject) raises -> PythonObject:
    """Computes the evaluation metrics for a given numerical solution in Mojo."""
    let builtins = Python.import_module("builtins")
    let np = Python.import_module("numpy")
    let numerics_testing = Python.import_module("specials._internal.numerics_testing")

    let result = elementwise[func](x, y)
    let msecs = run_benchmark[func](x, y).mean["ms"]()
    let relerr = numerics_testing.py_relative_error(
        tensor_to_numpy_array(result), truth
    )

    let report = builtins.list()
    _ = report.append(solution_name)
    _ = report.append(np.max(relerr))
    _ = report.append(np.mean(relerr))
    _ = report.append(msecs)

    return report


In [13]:
%%python
from timeit import timeit

import mpmath as mp

from specials._internal import numerics_testing

def py_mpmath_lbeta(x, y):
    """Computes the log-beta function using `mpmath`."""
    def _mp_lbeta_impl(a, b):
        a = mp.mpf(a)
        b = mp.mpf(b)
        with mp.workdps(30):
            res = mp.log(mp.beta(a, b))
        return res

    dtype = np.result_type(x, y)
    return np.frompyfunc(_mp_lbeta_impl, 2, 1)(x, y).astype(dtype)


def py_benchmark(func, *args):
    """Computes the average execution time of a Python function."""
    # Warmup phase
    _ = timeit(lambda: func(*args), number=2)

    msecs = 1000 * timeit(lambda: func(*args), number=100) / 100
    return msecs


def py_solution_report(solution_name, func, x_arr, y_arr, truth):
    """Computes the evaluation metrics for a given numerical solution in Python."""
    result = func(x_arr, y_arr)
    msecs = py_benchmark(func, x_arr, y_arr)
    relerr = numerics_testing.py_relative_error(result, truth)

    return [solution_name, np.max(relerr), np.mean(relerr), msecs]


#### 2.1.3. Baselines

We compare the log-beta function implementation in Specials with the following alternative solutions:
- **Naive**: It is an implementation based solely on the standard library function `math.lgamma`.
- **SciPy**: [SciPy](https://scipy.org/) is considered one of the leading Python libraries for scientific computing. It provides the `betaln` function, which is a [NumPy UFunc](https://numpy.org/devdocs/user/basics.ufuncs.html#ufuncs-basics) that wraps a C implementation available in the Cephes Mathematical Library [[link](https://netlib.org/cephes/)].
- **TFP[numpy]**: [TensorFlow Probability](https://www.tensorflow.org/probability) is a Python library in the TensorFlow ecosystem focused on deep probabilistic models. It provides an implementation for the log-beta function, `lbeta`, that is based on the method proposed in [[1](#didonato1992)]. In the library's `numpy` substrate, this function is defined using NumPy operators.


In [14]:
fn naive_lbeta[
    dtype: DType, simd_width: Int
](x: SIMD[dtype, simd_width], y: SIMD[dtype, simd_width]) -> SIMD[dtype, simd_width]:
    """Computes the log-beta function using the naive implementation."""
    return math.lgamma(x) + math.lgamma(y) - math.lgamma(x + y)


In [15]:
%%python
import scipy


py_scipy_lbeta = scipy.special.betaln
py_tfp_lbeta = tfp.math.lbeta


#### 2.1.4. Results

Let's run the experiment and print the results.

In [16]:
%%python
from tabulate import tabulate, SEPARATING_LINE


def py_print_table(data, domain_names, num_solutions):
    """Prints the evaluation metrics for all numerical solutions."""
    headers = [
        "\nDomain",
        "\nSolution",
        "Maximum\nRelative Error",
        "Mean\nRelative Error",
        "Mean Execution Time\n(in milliseconds)",
    ]

    # Insert domain names
    current_domain = 0
    for i, report in enumerate(data):
        if i % num_solutions == 0:
            data[i].insert(0, domain_names[current_domain])
            current_domain += 1
        else:
            data[i].insert(0, "")

    # Insert horizontal lines between domains
    for index in range(num_solutions, len(data) + num_solutions, num_solutions + 1):
        data.insert(index, SEPARATING_LINE)

    floatfmt = (".2e", ".2e", ".2e", ".2e", ".3f")
    table = tabulate(data, headers, tablefmt="simple", floatfmt=floatfmt)

    print(table)


In [17]:
import random

let builtins = Python.import_module("builtins")

random.seed(42)

let domain_name = String("0,")
domain_names = builtins.list()

var data = builtins.list()

print("Running the experiment. This may take a while...\n")

for i in range(max_values.__len__()):
    let max_value = max_values[i]

    domain_names.append(domain_name + max_value.to_int())
    let a = random_uniform[dtype](min_value, max_value, num_samples)
    let b = random_uniform[dtype](min_value, max_value, num_samples)
    let a_arr = tensor_to_numpy_array(a)
    let b_arr = tensor_to_numpy_array(b)

    # mpmath
    let truth = py_mpmath_lbeta(a_arr, b_arr)

    # Specials
    let specials_report = solution_report["Specials", specials.lbeta, dtype](
        a, b, truth
    )
    data.append(specials_report)

    # Naive
    let naive_report = solution_report["Naive", naive_lbeta, dtype](a, b, truth)
    data.append(naive_report)

    # SciPy
    let scipy_report = py_solution_report("SciPy", py_scipy_lbeta, a_arr, b_arr, truth)
    data.append(scipy_report)

    # TensorFlow Probability, NumPy substrate
    let tfp_report = py_solution_report("TFP[numpy]", py_tfp_lbeta, a_arr, b_arr, truth)
    data.append(tfp_report)

py_print_table(data, domain_names, 4)


Running the experiment. This may take a while...

                               Maximum              Mean    Mean Execution Time
Domain    Solution      Relative Error    Relative Error      (in milliseconds)
--------  ----------  ----------------  ----------------  ---------------------
0,10      Specials            6.65e-12          7.97e-16                  3.297
          Naive               8.02e-12          1.25e-15                  2.458
          SciPy               2.13e-11          1.50e-15                  4.651
          TFP[numpy]          1.43e-11          9.85e-16                 15.169
--------  ----------  ----------------  ----------------  ---------------------
0,100     Specials            4.75e-14          5.70e-17                  2.594
          Naive               1.42e-12          1.32e-15                  2.279
          SciPy               1.60e-12          6.61e-16                  7.166
          TFP[numpy]          6.63e-14          7.31e-17              

Regarding the maximum relative error, the Specials implementation shows the best results, except in the domain (0, 1000], where the TensorFlow Probability implementation demonstrates higher accuracy. As for the mean relative error, Specials outperforms the competitors unanimously.

In terms of the mean execution time, the naive, Specials, and SciPy implementations are in the same order of magnitude. Overall, the naive implementation exhibits the best performance, followed by Specials, and then SciPy. In all evaluated domains, the TensorFlow Probability implementation has shown to be an order of magnitude inferior to the other solutions.

## References

[<a id="didonato1992">1</a>]
Didonato, A. R., & Morris Jr, A. H. (1992). Algorithm 708: Significant digit computation of the incomplete beta function ratios. _ACM Transactions on Mathematical Software (TOMS)_, 18(3), 360-373. [[Link](https://dl.acm.org/doi/10.1145/131766.131776)]

[<a id="machler">2</a>]
Mächler, M. Computing the Beta Function for Large Arguments. [[Link](https://cran.r-project.org/web/packages/DPQ/vignettes/comp-beta.pdf)]

## Appendix A: System information

Below, information about the system used to run the experiment.

In [18]:
%%python

subprocess.run(["modular", "-v"])
subprocess.run(["mojo", "-v"])


modular 0.2.0 (355ea9c0)
mojo 0.5.0 (6e50a738)


In [19]:
from runtime.llcl import num_cores
from sys.info import (
    os_is_linux,
    os_is_windows,
    os_is_macos,
    has_sse4,
    has_avx,
    has_avx2,
    has_avx512f,
    has_vnni,
    has_neon,
    is_apple_m1,
    has_intel_amx,
    _current_target,
    _current_cpu,
    _triple_attr,
)

let os: StringLiteral
if os_is_linux():
    os = "linux"
elif os_is_macos():
    os = "macOS"
else:
    os = "windows"

let cpu = String(_current_cpu())
let arch = String(_triple_attr())

var cpu_features = String("")
if has_sse4():
    cpu_features += " sse4"
if has_avx():
    cpu_features += " avx"
if has_avx2():
    cpu_features += " avx2"
if has_avx512f():
    cpu_features += " avx512f"
if has_vnni():
    if has_avx512f():
        cpu_features += " avx512_vnni"
    else:
        cpu_features += " avx_vnni"
if has_intel_amx():
    cpu_features += " intel_amx"
if has_neon():
    cpu_features += " neon"
if is_apple_m1():
    cpu_features += " apple_m1"

if len(cpu_features) > 0:
    cpu_features = cpu_features[1:]

print("System Information")
print("    OS          :", os)
print("    CPU         :", cpu)
print("    Arch        :", arch)
print("    Num Cores   :", num_cores())
print("    CPU Features:", cpu_features)


System Information
    OS          : linux
    CPU         : haswell
    Arch        : x86_64-unknown-linux-gnu
    Num Cores   : 4
    CPU Features: sse4 avx avx2


In [20]:
%%python
import pkg_resources
import sys

def get_version(package):
    """Returns the version of a Python package."""
    return pkg_resources.get_distribution(package).version

print("mpmath version:", mp.__version__)
print("NumPy version:", np.__version__)
print("Python version:", sys.version)
print("SciPy version:", scipy.__version__)
print("Tabulate version:", get_version("tabulate"))
print("TensorFlow version:", get_version("tensorflow"))
print("TensorFlow Probability version:", tfp.__version__)


mpmath version: 1.3.0
NumPy version: 1.26.0
Python version: 3.11.5 (main, Sep 11 2023, 14:07:11) [GCC 11.2.0]
SciPy version: 1.11.3
Tabulate version: 0.9.0
TensorFlow version: 2.15.0
TensorFlow Probability version: 0.22.1
